In [311]:
#D:\DI\zhou
import pandas as pd
import numpy as np
from datetime import datetime
import sklearn
import statsmodels.api as sm
from itertools import combinations

In [3]:

df = pd.read_excel("D:\DI\zhou\contact.xlsx")
#data cleaning and preprocessing:
#check data first
#create number of listing (booked), see if relative to dim_total_reviews
#calculate time ahead of checking when booking
#time of staying
#create booking success, based on if ts_accepted_at_first is not null
#fill m_first_message_length_in_characters nan to 0 and m_guest_first to average value
#filter book_it only
#filter m_guests_first <= dim_person_capacity


In [111]:
#firstly check data integrity 
#sum(df["ds_checkin_first"].isnull())
for i in range(len(df.keys())):
    if sum(df[df.keys()[i]].isnull()) != 0:
        print (df.keys()[i])
        print (sum(df[df.keys()[i]].isnull()))

# results:
# ts_reply_at_first
# 1392
# ts_accepted_at_first
# 6398
# ts_booking_at
# 6452
# m_guests_first
# 37
# m_first_message_length_in_characters
# 2226
# dim_guest_language
# 23143


ts_reply_at_first
1392
ts_accepted_at_first
6398
ts_booking_at
6452
m_guests_first
37
m_first_message_length_in_characters
2226
dim_guest_language
23143


In [194]:
df_1 = df.copy()
#to add column of total listing number for each id
list_num_count = []
#list for count of each listing id 
listing_count = df["id_listing_anon"].value_counts()
for i in range(len(df["id_listing_anon"])):
    list_num_count.append(listing_count[df["id_listing_anon"][i]])
df_1['listing_num'] = pd.Series(list_num_count, index=df_1.index)
#check
df_1.head()

,id_guest_anon,id_host_anon,id_listing_anon,ts_interaction_first,ts_reply_at_first,ts_accepted_at_first,ts_booking_at,ds_checkin_first,ds_checkout_first,m_guests_first,m_interactions,m_first_message_length_in_characters,dim_contact_channel_first,dim_room_type,dim_total_reviews,dim_person_capacity,dim_guest_language,dim_host_language,listing_num
0,56d70d7c-1d0a-4594-a250-ed62f7cf7ac4,65a56b50-faf2-44a2-845f-0c467f41d720,4deeb033-183e-437c-b94c-851ffee8cec8,2013-01-13 21:03:07,2013-01-14 21:19:42,2013-01-14 21:19:42,2013-01-14 21:19:42,2013-01-18,2013-01-20,2.0,13,165.0,book_it,Private room,78,2,NaN,es,33
1,dcfb93c4-07ca-41e8-b134-89b49ddc7a36,65a56b50-faf2-44a2-845f-0c467f41d720,4deeb033-183e-437c-b94c-851ffee8cec8,2013-01-15 23:43:49,2013-01-16 09:04:37,2013-01-16 09:04:37,2013-01-16 09:04:37,2013-01-23,2013-01-25,2.0,5,NaN,book_it,Private room,78,2,NaN,es,33
2,aef63ee0-7fd8-449f-867d-e710c39694e1,65a56b50-faf2-44a2-845f-0c467f41d720,4deeb033-183e-437c-b94c-851ffee8cec8,2013-03-19 21:30:13,2013-03-20 12:50:49,2013-03-20 12:50:49,2013-03-20 12:50:49,2013-05-31,2013-06-03,2.0,7,350.0,book_it,Private room,78,2,NaN,es,33
3,58788b6f-1215-40a5-8fb0-75d2e399880c,65a56b50-faf2-44a2-845f-0c467f41d720,4deeb033-183e-437c-b94c-851ffee8cec8,2013-01-03 16:32:01,2013-01-03 17:02:13,2013-01-03 17:02:53,2013-01-03 17:02:53,2013-02-09,2013-02-11,2.0,5,426.0,book_it,Private room,78,2,NaN,es,33
4,c3945c48-53b6-4e19-8bda-7d5b51d14ec1,65a56b50-faf2-44a2-845f-0c467f41d720,4deeb033-183e-437c-b94c-851ffee8cec8,2013-04-13 04:31:33,2013-04-13 16:43:53,2013-04-13 16:44:20,2013-04-13 16:44:20,2013-06-06,2013-06-08,2.0,7,239.0,book_it,Private room,78,2,NaN,es,33


In [146]:
#check correlation between review number and listing number
check_list = []
check_lnum = []
check_rnum = []
for i in range(len(df["id_listing_anon"])):
    if df_1["id_listing_anon"][i] not in check_list:
        check_list.append(df_1["id_listing_anon"][i])
        check_lnum.append(float(df_1["listing_num"][i]))
        check_rnum.append(float(df_1["dim_total_reviews"][i]))


#data = df_1[['listing_num','dim_total_reviews']]
print (len(check_lnum),len(check_rnum))
np.corrcoef(check_lnum, check_rnum)
#correlation is ~0.80 rather highly correlated.

3172 3172


array([[ 1.        ,  0.79765684],
       [ 0.79765684,  1.        ]])

In [223]:
#create entry of time intervel between booking and checking
time_ahead = []
time_stay = []
for i in range(len(df_1["id_listing_anon"])):
#     t_inter = datetime.strptime(df_2["ts_interaction_first"][i], "%m/%d/%Y %I:%M:%S %p")
#     t_in = datetime.strptime(df_2["ds_checkin_first"][i], "%m/%d/%Y")
#     t_out = datetime.strptime(df_2["ds_checkout_first"][i], "%m/%d/%Y")
    try:
        t_inter = df_1["ts_interaction_first"][i]
        t_in = df_1["ds_checkin_first"][i]
        t_out = df_1["ds_checkout_first"][i]
    
    
    
        time_ahead.append((t_in - t_inter).days)
        time_stay.append((t_out - t_in).days) 
    except:
        print (i)

df_1['time_ahead'] = pd.Series(time_ahead, index=df_1.index)        
df_1['time_stay'] = pd.Series(time_stay, index=df_1.index)

In [169]:
#"1/6/2014"-"12/25/2013"
dt1 = datetime.strptime("1/6/2014", "%m/%d/%Y")
dt2 = datetime.strptime("12/25/2013 7:14:56 PM","%m/%d/%Y %I:%M:%S %p")
c= dt1-dt2

In [199]:
#df_2["ts_interaction_first"][1412]
#df_2["ds_checkin_first"][1412]
#df_2["ds_checkout_first"][1412]
for i in range(len(df_1["id_listing_anon"])):
#     t_inter = datetime.strptime(df_2["ts_interaction_first"][i], "%m/%d/%Y %I:%M:%S %p")
#     t_in = datetime.strptime(df_2["ds_checkin_first"][i], "%m/%d/%Y")
#     t_out = datetime.strptime(df_2["ds_checkout_first"][i], "%m/%d/%Y")
    try:
        t_inter = df_1["ts_interaction_first"][i]
        t_in = df_1["ds_checkin_first"][i]
        t_out = df_1["ds_checkout_first"][i]
    
    
    
        time_ahead.append((t_in - t_inter).days)
        time_stay.append((t_out - t_in).days) 
    except:
        print (i)

In [268]:
#convert ts_accepted_at_first to acceptance
not_accept = df_1["ts_accepted_at_first"].isnull()
df_1['accept'] = pd.Series(-not_accept, index=df_temp.index)
#fill Nan in  m_first_message_length_in_characters, and m_guest_staying as average
df_1["m_first_message_length_in_characters"]= df_1["m_first_message_length_in_characters"].fillna(value=0)
df_1["m_guests_first"] = df_1["m_guests_first"].fillna(value=df_1["m_guests_first"].mean())


In [274]:
#filter out only book_it (already checked only two types in data)
book_num = df_1["dim_contact_channel_first"] == "book_it"
df_2 = df_1[book_num]
print (len(df_2))
guest_num = df_2["m_guests_first"] <= df_2["dim_person_capacity"]
df_3 = df_2[guest_num]
print (len(df_3))
path = 'D:\\DI\\zhou\\'
df_3.to_csv(path+'try.csv')

19858
19461


In [327]:
#now start model building, inputs use subsets of "m_guests_first","m_interactions","m_first_message_length_in_characters","dim_total_reviews","dim_person_capacity"\
#,"time_ahead","listing_num"
#this is a loose condition, try it first
#apply best subset selection combined with LR

selection_list = ["m_guests_first","m_interactions","m_first_message_length_in_characters","dim_total_reviews",\
                  "dim_person_capacity","time_ahead","time_stay"]
best_score = 0
test = np.random.rand(len(df_3)) < 0.8
min_max_scaler = preprocessing.MinMaxScaler()
kfold = model_selection.KFold(n_splits=10,random_state=7)
modelCV = sklearn.linear_model.LogisticRegression()
for i in range(len(selection_list)):
    for combo in combinations(selection_list, i+1): 
        x = df_3[list(combo)]
        x_norm = min_max_scaler.fit_transform(x)
        x_norm_test = x_norm[test]
        results = model_selection.cross_val_score(modelCV, x_norm_test, df_3[test]["accept"], cv=kfold, scoring = "accuracy")
        if results.mean() > best_score:
            best_score = results.mean()
            best_comb = combo
print(best_comb)
print(best_score)

#results:
# ('m_interactions', 'dim_total_reviews', 'time_ahead')
# 0.807728010703

('m_interactions', 'dim_total_reviews', 'time_ahead')
0.807728010703


In [347]:
#now from best subset, select highest impact term :
test_list = ['m_interactions', 'dim_total_reviews', 'time_ahead']
for i in range(len(test_list)):
    list_use = test_list[:]
    del list_use[i]
    x = df_3[list_use]
    x_norm = min_max_scaler.fit_transform(x)
    x_norm_test = x_norm[test]
    results = model_selection.cross_val_score(modelCV, x_norm_test, df_3[test]["accept"], cv=kfold, scoring = "accuracy")
    print (i)
    print (results.mean())
#results:
# 0
# 0.704082712393
# 1
# 0.78478972388
# 2
# 0.806500620523
#test if fit individually which one has highest accuracy
test_list = ['m_interactions', 'dim_total_reviews', 'time_ahead']
for i in range(len(test_list)):
    a = list(str(test_list[i]))
    x = df_3[[test_list[i]]]
    x_norm = min_max_scaler.fit_transform(x)
    x_norm_test = x_norm[test]
    results = model_selection.cross_val_score(modelCV, x_norm_test, df_3[test]["accept"], cv=kfold, scoring = "accuracy")
    print (i)
    print (results.mean())
#resuts:
# 0
# 0.786405420845
# 1
# 0.696072376476
# 2
# 0.68224370249

#therefore m_interactions is most important feature

0
0.704082712393
1
0.78478972388
2
0.806500620523
0
0.786405420845
1
0.696072376476
2
0.68224370249


In [368]:
x = df_3[list(best_comb)]
x_norm = min_max_scaler.fit_transform(x)
x_norm_test = x_norm[test]
results = model_selection.cross_val_score(modelCV, x_norm_test, df_3[test]["accept"], cv=kfold, scoring = "accuracy")

In [369]:
modelCV.fit(x_norm_test,df_3[test]["accept"])
sum(modelCV.predict(x_norm[~test]) == df_3[~test]["accept"])/len(x_norm[~test])
#test score 0.799

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [378]:
#2 part, message experiment. 
experiment_list = pd.read_excel("D:/DI/zhou/assignment.xlsx")


In [382]:
experiment_list['ab'].value_counts()

control      10029
treatment     9967
Name: ab, dtype: int64

In [401]:
cont = experiment_list['ab'] == "control"
experiment_list_control = experiment_list[cont]
experiment_list_treatment = experiment_list[~cont]
a = experiment_list_control['id_user_anon'].isin(experiment_list_treatment['id_user_anon'])
#experiment_list_control['id_user_anon'].value_counts()
sum(a)
#results not 0 indicating there is overlap

9110

In [423]:
#indexing
index_control = df_1['id_guest_anon'].isin(experiment_list_control['id_user_anon']) & -df_1['id_guest_anon'].isin(experiment_list_treatment['id_user_anon'])
print (sum(index_control))
index_treatment = -df_1['id_guest_anon'].isin(experiment_list_control['id_user_anon']) & df_1['id_guest_anon'].isin(experiment_list_treatment['id_user_anon'])
print (sum(index_treatment))
index_both = df_1['id_guest_anon'].isin(experiment_list_control['id_user_anon']) & df_1['id_guest_anon'].isin(experiment_list_treatment['id_user_anon'])
print (sum(index_both))
print (len(df_1))
#checked total number

10998
10961
1184
23143


In [424]:
#cleaning
df_control_1 = df_1[index_control]
df_treatment_1 = df_1[index_treatment]

book_control = df_control_1["dim_contact_channel_first"] == "book_it"
book_treatment = df_treatment_1["dim_contact_channel_first"] == "book_it"

df_control_2 = df_control_1[book_control]
df_treatment_2 = df_treatment_1[book_treatment]

guest_control = df_control_2["m_guests_first"] <= df_control_2["dim_person_capacity"]
guest_treatment = df_treatment_2["m_guests_first"] <= df_treatment_2["dim_person_capacity"]
df_control_3 = df_control_2[guest_control]
df_treatment_3 = df_treatment_2[guest_treatment]

path = 'D:\\DI\\zhou\\'
df_control_3.to_csv(path+'control_cleaned.csv')
df_treatment_3.to_csv(path+'treatment_cleaned.csv')



In [425]:
print (sum(df_control_3['accept'])/len(df_control_3))
print (sum(df_treatment_3['accept'])/len(df_treatment_3))
# rates doesn't vary much but can be deceive for m_first_message_length_in_characters can also contribute to rates

0.686111710157
0.677591463415
